# This notebook serves a test workspace. For a more robust testing space, please check out the other notebook in this repo. This notebook is devoted to tensorflow, and trying to implement ML at a lower level. 

In [6]:
#import dependencies, and dataset. 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline



In [7]:
df = pd.read_csv('../splits_optimizer/model_ready_data.csv')

In [8]:
df.head()

,Unnamed: 0,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,...,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,home_SPREAD,PLUS_MINUS,Home Team Won?
0,0,41.033333,8.966667,17.366667,35.066667,25.966667,7.366667,14.200000,5.766667,108.400000,...,13.600000,3.766667,108.033333,1.916725,0.333378,0.458320,0.766028,7.0,19,1
1,1,35.633333,6.366667,15.600000,29.066667,19.800000,7.300000,14.233333,5.033333,93.233333,...,13.500000,4.633333,105.300000,1.909397,0.333049,0.465288,0.719054,-14.5,8,1
2,2,39.466667,8.533333,16.466667,30.700000,23.233333,9.533333,14.133333,4.700000,103.933333,...,13.533333,2.866667,106.433333,1.954478,0.337522,0.429603,0.794908,-4.0,-2,0
3,3,34.866667,5.033333,18.500000,33.233333,19.766667,6.333333,12.700000,5.366667,93.266667,...,14.366667,4.966667,93.133333,1.579379,0.346225,0.434064,0.771320,-2.5,3,1
4,4,39.000000,9.566667,15.866667,32.700000,25.800000,8.300000,14.600000,4.466667,103.433333,...,15.000000,4.566667,95.233333,1.347540,0.345794,0.454824,0.770221,-3.0,-1,0


In [9]:
outcome = df['PLUS_MINUS'] #df['home_SPREAD'] + df['PLUS_MINUS'] 

In [10]:
y = []
for val in outcome:
    if val>0: 
        y.append(1) #home team wins. 
    else:
        y.append(0)
        
del df['PLUS_MINUS']


X = df
del X['Unnamed: 0']
del X['Home Team Won?']

In [11]:
hometeamwins = sum(y)/len(y)  
print("The home team wins ", 100*hometeamwins, "% of the time. ")

The home team wins  59.038961038961034 % of the time. 


In [12]:
#ready for data normalization and splitting


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#Split all into training and testing
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size = .25)
#Set aside some training data for validation. 
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,shuffle=False,test_size = .25)

scaler = MinMaxScaler()
#Normalization of dataset. 
scaler.fit(X_train.values)
X_train = scaler.transform(X_train.values)
X_val = scaler.transform(X_val.values)
X_test = scaler.transform(X_test.values)


In [13]:
#import tensorflow
import  tensorflow as tf
tf.set_random_seed(456)




In [14]:
# Generate tensorflow graph
d = X_train.shape[1]
n_hidden = 100
learning_rate = .001
n_epochs = 100
batch_size = 50
dropout_prob = .5


with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, d))
    y = tf.placeholder(tf.float32, (None,))
    keep_prob = tf.placeholder(tf.float32)
with tf.name_scope("hidden-layer-1"):
    W = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden_1 = tf.nn.sigmoid(tf.matmul(x, W) + b)
  # Apply dropout
    x_hidden_1 = tf.nn.dropout(x_hidden_1, keep_prob)

with tf.name_scope("hidden-layer-2"):
    W = tf.Variable(tf.random_normal((n_hidden, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden_2 = tf.nn.sigmoid(tf.matmul(x_hidden_1, W) + b)
  # Apply dropout
    x_hidden_2 = tf.nn.dropout(x_hidden_2, keep_prob)

with tf.name_scope("output"):
    W = tf.Variable(tf.random_normal((n_hidden, 1)))
    b = tf.Variable(tf.random_normal((1,)))
    y_logit = tf.matmul(x_hidden_2, W) + b
  # the sigmoid gives the class probability of 1
    y_one_prob = tf.sigmoid(y_logit)
  # Rounding P(y=1) will give the correct prediction.
    y_pred = tf.round(y_one_prob)
with tf.name_scope("loss"):
  # Compute the cross-entropy term for each datapoint
    y_expand = tf.expand_dims(y, 1)
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
  # Sum all contributions
    l = tf.reduce_sum(entropy)

with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()

In [15]:

#Write to tensorboard
train_writer = tf.summary.FileWriter('/tmp/nba-tf-neural-network',tf.get_default_graph())


In [16]:
#Begin iteration through batches of training data and see model improve. 
N = X_train.shape[0]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 0
    for epoch in range(n_epochs):
        pos = 0
        while pos < N:
            batch_X = X_train[pos:pos+batch_size]
            batch_y = y_train[pos:pos+batch_size]
            feed_dict = {x: batch_X, y: batch_y, keep_prob: dropout_prob}
            _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
            train_writer.add_summary(summary, step)

            step += 1
            pos += batch_size
  # Make Predictions (set keep_prob to 1.0 for predictions)
        train_y_pred = sess.run(y_pred, feed_dict={x: X_train, keep_prob: 1.0})
       # val_y_pred = sess.run(y_pred, feed_dict={x: X_val, keep_prob: 1.0})
        #print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
        print("training acc. ", accuracy_score(y_train,train_y_pred), "%")
      #  print("validation acc. ", accuracy_score(y_val,val_y_pred), "%")
    test_y_pred = sess.run(y_pred, feed_dict={x: X_test, keep_prob: 1.0})

train_score = accuracy_score(y_train, train_y_pred)
print("Train Classification Accuracy: %f" % train_score)
#valid_weighted_score = accuracy_score(valid_y, valid_y_pred, sample_weight=valid_w)
#print("Valid Weighted Classification Accuracy: %f" % valid_weighted_score)
test_score = accuracy_score(y_test, test_y_pred)
print("Test Classification Accuracy: %f" % test_score)

training acc.  0.645727482679 %
training acc.  0.658198614319 %
training acc.  0.667898383372 %
training acc.  0.669284064665 %
training acc.  0.672979214781 %
training acc.  0.678983833718 %
training acc.  0.676674364896 %
training acc.  0.676674364896 %
training acc.  0.68129330254 %
training acc.  0.6896073903 %
training acc.  0.683140877598 %
training acc.  0.680369515012 %
training acc.  0.683140877598 %
training acc.  0.678060046189 %
training acc.  0.680369515012 %
training acc.  0.675750577367 %
training acc.  0.673903002309 %
training acc.  0.673903002309 %
training acc.  0.672979214781 %
training acc.  0.678060046189 %
training acc.  0.674826789838 %
training acc.  0.672055427252 %
training acc.  0.671593533487 %
training acc.  0.677598152425 %
training acc.  0.683602771363 %
training acc.  0.68545034642 %
training acc.  0.688221709007 %
training acc.  0.691454965358 %
training acc.  0.693302540416 %
training acc.  0.688683602771 %
training acc.  0.695612009238 %
training acc

In [17]:
!tensorboard --logdir=/tmp/nba-tf-neural-network

W1107 18:26:22.767660 Reloader tf_logging.py:121] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1107 18:26:22.768768 Reloader tf_logging.py:121] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1107 18:26:22.776395 Reloader tf_logging.py:121] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1107 18:26:22.777164 Reloader tf_logging.py:121] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
TensorBoard 1.7.0a20180302 at http://Noahs-MBP.fios-router.home:6006 (Press CTRL+C to quit)
W1107 18:26:27.124647 Reloader tf_logging.py:121] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph event

# Conclusion. The tensorflow model clearly improves, but leaves a lot left to desire. The difference of nearly 4% between training and testing suggests there is some overfitting put in place. To remedy this, regularization techniques such as cross validation and others are put in place in the next notebook. 

# as a bonus, I did notice something initially wrong with my tensorflow model. I've included a picture of the losses graph that helped me debug the issue, and is discussed further in the README. 